In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import warnings
import pandas as pd
import time
import os
from tqdm.notebook import tqdm
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from collections import Counter
from collections import defaultdict
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import normalize
import math
from konlpy.tag import Twitter
from nltk.tag import pos_tag
from nltk.tag import untag
from nltk import Text
from matplotlib import pyplot as plt
from nltk import FreqDist
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer
from glob import glob
from tqdm.notebook import tqdm




warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": "C:/Users/kimEn/Desktop", #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
driver = webdriver.Chrome('chromedriver.exe', options=options)

driver.get('https://paperswithcode.com/sota')

In [3]:
#맨처음 목록 목차 분류작업

class_2b_url=[]  #big 분류페이지로
class0=[] #class_big와 길이확인**

#browse sota대분류 class0 -  중분류 class1  소분류 -pdf설명페이지-pdf다운 길이확인ㅇ
#class0=맨처음페이지 대분류 리스트, class0_url=대분류페이지 이동url
list1=driver.find_elements(By.CLASS_NAME,'col-md-12 > h4 > a')
for i in list1:
    class0.append(i.text)
    class_2b_url.append(i.get_attribute('href'))

In [4]:
card_url=[]
see_all_url=[]

for i in range(len(class_2b_url)):
    driver.get(class_2b_url[i]) 
    
    card_url_elem=driver.find_elements(By.CLASS_NAME,'card > a')
    see_all_elem = driver.find_elements(By.CLASS_NAME,'sota-all-tasks > a')
    
    for c in card_url_elem:
        if c.get_attribute('href') not in card_url:
            card_url.append(c.get_attribute('href'))

    for s in see_all_elem:          
            see_all_url.append(s.get_attribute('href'))            


In [5]:
#중분류 모두보기 있는 경우 앞과 겹치지 않게 url 수집 mid_url에 저장

for i in range(len(see_all_url)):
    driver.get(see_all_url[i])
    card_url_elem2=driver.find_elements(By.CLASS_NAME,'card > a')

    for c2 in card_url_elem2:
        if c2.get_attribute('href') not in card_url:
            card_url.append(c2.get_attribute('href'))

save_url=card_url

In [7]:
# 논문 제목, 상세 url 수집
# classname>css section
import collections

title = []
url = []
pdf=[]
classification1=[]
classification2=[]

cla_dict=collections.defaultdict(list)
pdf_cla_dict={}


#mid_url(소분류 url) 반복하면서 title, url 리스트 생성
for i in range(len(card_url)): #now range of 10
    driver.get(card_url[i])
    

#     #scroll down - all page
#     last_height=driver.execute_script("return document.body.scrollHeight") #마지막 위치 저장

#     while True:
#         driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") 
#         time.sleep(3)
#         new_height=driver.execute_script("return document.body.scrollHeight") #새로운 위치 저장
#         if new_height==last_height:
#             break
#         last_height=new_height #새로운 위치를 마지막 위치에 저장

# scroll down for in range
#     last_height=driver.execute_script("return document.body.scrollHeight")
#     for i in range(3):
#         driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") 
#         time.sleep(3)
#         new_height=driver.execute_script("return document.body.scrollHeight") #새로운 위치 저장
#         if new_height==last_height:
#             break
#         last_height=new_height #새로운 위치를 마지막 위치에 저장

   
    #title, url
    Tlist=driver.find_elements(By.CLASS_NAME,'col-lg-9.item-content > h1 > a')
    classification1_elem=driver.find_elements(By.CLASS_NAME,'dataset-header > a')
    classification2_elem=driver.find_elements(By.CLASS_NAME, 'artefact-header > h1')
        
        
    for cla1 in classification1_elem:
        cla_item=cla1.get_attribute('href').split('/')[-1].replace('-',' ')
        classification1.append(cla_item)

        for cla2 in classification2_elem:
            classification2.append(cla2.text)
            cla_dict[cla_item].append(cla2.text)
            
            
    for i in Tlist:
        if i.get_attribute not in url and i.text not in title: 
            title.append(i.text)
            url.append(i.get_attribute('href'))

            

In [8]:
url_cla_dict={}

for i in range(len(card_url)): 
    driver.get(card_url[i])
    classification2_elem=driver.find_elements(By.CLASS_NAME, 'artefact-header > h1')
    Tlist=driver.find_elements(By.CLASS_NAME,'col-lg-9.item-content > h1 > a')
        
    for cla2 in classification2_elem:
        for i in Tlist:
            url_cla_dict[i.text]=cla2.text

Deep Residual Learning for Image Recognition Image Classification
Very Deep Convolutional Networks for Large-Scale Image Recognition Image Classification
MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications Image Classification
Densely Connected Convolutional Networks Image Classification
CSPNet: A New Backbone that can Enhance Learning Capability of CNN Image Classification
MobileNetV2: Inverted Residuals and Linear Bottlenecks Image Classification
Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization Image Classification
EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks Image Classification
Rethinking the Inception Architecture for Computer Vision Image Classification
An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale Image Classification
Distilling the Knowledge in a Neural Network Knowledge Distillation
Well-Read Students Learn Better: On the Importance of Pre-training Compact Mo

Deep Residual Learning for Image Recognition Object Detection
YOLOv3: An Incremental Improvement Object Detection
YOLO9000: Better, Faster, Stronger Object Detection
YOLOv4: Optimal Speed and Accuracy of Object Detection Object Detection
Focal Loss for Dense Object Detection Object Detection
SSD: Single Shot MultiBox Detector Object Detection
Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks Object Detection
Mask R-CNN Object Detection
MMDetection: Open MMLab Detection Toolbox and Benchmark Object Detection
You Only Look Once: Unified, Real-Time Object Detection Object Detection
Frustum PointNets for 3D Object Detection from RGB-D Data 3D Object Detection
VoxelNet: End-to-End Learning for Point Cloud Based 3D Object Detection 3D Object Detection
nuScenes: A multimodal dataset for autonomous driving 3D Object Detection
Deep Hough Voting for 3D Object Detection in Point Clouds 3D Object Detection
Complex-YOLO: Real-time 3D Object Detection on Point Clouds 3D 

Deep Residual Learning for Image Recognition Image-to-Image Translation
Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks Image-to-Image Translation
Image-to-Image Translation with Conditional Adversarial Networks Image-to-Image Translation
StarGAN: Unified Generative Adversarial Networks for Multi-Domain Image-to-Image Translation Image-to-Image Translation
U-GAT-IT: Unsupervised Generative Attentional Networks with Adaptive Layer-Instance Normalization for Image-to-Image Translation Image-to-Image Translation
Semantic Image Synthesis with Spatially-Adaptive Normalization Image-to-Image Translation
High-Resolution Image Synthesis and Semantic Manipulation with Conditional GANs Image-to-Image Translation
Multimodal Unsupervised Image-to-Image Translation Image-to-Image Translation
Everybody Dance Now Image-to-Image Translation
StarGAN v2: Diverse Image Synthesis for Multiple Domains Image-to-Image Translation
Image Inpainting for Irregular Holes Using Part

DeepFaceLab: Integrated, flexible and extensible face-swapping framework Face Swapping
FaceForensics++: Learning to Detect Manipulated Facial Images Face Swapping
FaceShifter: Towards High Fidelity And Occlusion Aware Face Swapping Face Swapping
The DeepFake Detection Challenge (DFDC) Dataset Face Swapping
MesoNet: a Compact Facial Video Forgery Detection Network Face Swapping
Celeb-DF: A Large-scale Challenging Dataset for DeepFake Forensics Face Swapping
In Ictu Oculi: Exposing AI Generated Fake Face Videos by Detecting Eye Blinking Face Swapping
Exposing DeepFake Videos By Detecting Face Warping Artifacts Face Swapping
Face X-ray for More General Face Forgery Detection Face Swapping
FDFtNet: Facing Off Fake Images using Fake Detection Fine-tuning Network Face Swapping
Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network Image Super-Resolution
Perceptual Losses for Real-Time Style Transfer and Super-Resolution Image Super-Resolution
Image Super-Resolut

Visual-Inertial Monocular SLAM with Map Reuse Simultaneous Localization and Mapping
Kimera: an Open-Source Library for Real-Time Metric-Semantic Localization and Mapping Simultaneous Localization and Mapping
Sparse-to-Dense: Depth Prediction from Sparse Depth Samples and a Single Image Simultaneous Localization and Mapping
ORB-SLAM2: an Open-Source SLAM System for Monocular, Stereo and RGB-D Cameras Simultaneous Localization and Mapping
Online Spatial Concept and Lexical Acquisition with Simultaneous Localization and Mapping Simultaneous Localization and Mapping
Incremental Visual-Inertial 3D Mesh Generation with Structural Regularities Simultaneous Localization and Mapping
LiDARTag: A Real-Time Fiducial Tag System for Point Clouds Simultaneous Localization and Mapping
3D Dynamic Scene Graphs: Actionable Spatial Perception with Places, Objects, and Humans Simultaneous Localization and Mapping
DXSLAM: A Robust and Efficient Visual SLAM System with Deep Features Simultaneous Localization

...

Rank consistent ordinal regression for neural networks with application to age estimation Age And Gender Classification
Age and Gender Classification using Convolutional Neural Networks Age And Gender Classification
Increasingly Packing Multiple Facial-Informatics Modules in A Unified Deep-Learning Model via Lifelong Learning Age And Gender Classification
Quantifying Facial Age by Posterior of Age Comparisons Age And Gender Classification
Age Group and Gender Estimation in the Wild with Deep RoR Architecture Age And Gender Classification
A Fusion-based Gender Recognition Method Using Facial Images Age And Gender Classification
007: Democratically Finding The Cause of Packet Drops Age And Gender Classification
Joint Estimation of Age and Gender from Unconstrained Face Images using Lightweight Multi-task CNN for Mobile Applications Age And Gender Classification
Multimodal Age and Gender Classification Using Ear and Profile Face Images Age And Gender Classification
Compacting, Picking and

HalluciNet-ing Spatiotemporal Representations Using a 2D-CNN Action Recognition In Still Images
Pyramid Stereo Matching Network Stereo-LiDAR Fusion
End-to-End Learning of Geometry and Context for Deep Stereo Regression Stereo-LiDAR Fusion
3D LiDAR and Stereo Fusion using Stereo Matching Network with Conditional Cost Volume Normalization Stereo-LiDAR Fusion
Learning Guided Convolutional Network for Depth Completion Stereo-LiDAR Fusion
Scene Completeness-Aware Lidar Depth Completion for Driving Scenario Stereo-LiDAR Fusion
Non-Local Spatial Propagation Network for Depth Completion Stereo-LiDAR Fusion
AI Playground: Unreal Engine-based Data Ablation Tool for Deep Learning Indoor Monocular Depth Estimation
DepthLab: Real-Time 3D Interaction With Depth Maps for Mobile Augmented Reality Indoor Monocular Depth Estimation
Learning to Recover 3D Scene Shape from a Single Image Indoor Monocular Depth Estimation
Deep Attentional Guided Image Filtering Depth Image Upsampling
PHOCNet: A Deep Convol

The Liver Tumor Segmentation Benchmark (LiTS) Liver Segmentation
Med3D: Transfer Learning for 3D Medical Image Analysis Liver Segmentation
Models Genesis: Generic Autodidactic Models for 3D Medical Image Analysis Liver Segmentation
Learning Semantics-enriched Representation via Self-discovery, Self-classification, and Self-restoration Liver Segmentation
H-DenseUNet: Hybrid Densely Connected UNet for Liver and Tumor Segmentation from CT Volumes Liver Segmentation
Imperfect Segmentation Labels: How Much Do They Matter? Liver Segmentation
Transformation Consistent Self-ensembling Model for Semi-supervised Medical Image Segmentation Liver Segmentation
Fully Automatic Liver Attenuation Estimation Combing CNN Segmentation and Morphological Operations Liver Segmentation
Generating large labeled data sets for laparoscopic image processing tasks using unpaired image-to-image translation Liver Segmentation
Optimal input configuration of dynamic contrast enhanced MRI in convolutional neural netwo

V-Net: Fully Convolutional Neural Networks for Volumetric Medical Image Segmentation Volumetric Medical Image Segmentation
On the Compactness, Efficiency, and Representation of 3D Convolutional Networks: Brain Parcellation as a Pretext Task Volumetric Medical Image Segmentation
Conditional Random Fields as Recurrent Neural Networks for 3D Medical Imaging Segmentation Volumetric Medical Image Segmentation
3D Densely Convolutional Networks for VolumetricSegmentation Volumetric Medical Image Segmentation
KiU-Net: Overcomplete Convolutional Architectures for Biomedical Image and Volumetric Segmentation Volumetric Medical Image Segmentation
Positional Contrastive Learning for Volumetric Medical Image Segmentation Volumetric Medical Image Segmentation
nnFormer: Interleaved Transformer for Volumetric Segmentation Volumetric Medical Image Segmentation
A Volumetric Transformer for Accurate 3D Tumor Segmentation Volumetric Medical Image Segmentation
U-Net: Convolutional Networks for Biomedical I

KeyboardInterrupt: 

In [61]:
#pdf_url 수집
for i in range(len(url)):    
        
    driver.get(url[i])
    pdf_url=driver.find_elements(By.CLASS_NAME,'badge.badge-light')[0].get_attribute('href')
    
    if pdf_url not in pdf:
        pdf.append(pdf_url)

In [62]:
df=pd.DataFrame(columns=['title','url','pdf'])

df['title']=title
df['url']=url
df['pdf']=pdf

In [63]:
#내용저장 csv
df.to_csv("theory.csv",header=True, index=False)
df_theory=pd.read_csv('theory.csv')
df_theory

driver.close()

,title,url,pdf
0,Deep Residual Learning for Image Recognition,https://paperswithcode.com/paper/deep-residual...,https://arxiv.org/pdf/1512.03385v1.pdf
1,Very Deep Convolutional Networks for Large-Sca...,https://paperswithcode.com/paper/very-deep-con...,https://arxiv.org/pdf/1409.1556v6.pdf
2,MobileNets: Efficient Convolutional Neural Net...,https://paperswithcode.com/paper/mobilenets-ef...,https://arxiv.org/pdf/1704.04861v1.pdf
3,Densely Connected Convolutional Networks,https://paperswithcode.com/paper/densely-conne...,https://arxiv.org/pdf/1608.06993v5.pdf
4,CSPNet: A New Backbone that can Enhance Learni...,https://paperswithcode.com/paper/cspnet-a-new-...,https://arxiv.org/pdf/1911.11929v1.pdf
...,...,...,...
89,Learning Pixel-level Semantic Affinity with Im...,https://paperswithcode.com/paper/learning-pixe...,https://arxiv.org/pdf/1803.10464v2.pdf
90,Integral Object Mining via Online Attention Ac...,https://paperswithcode.com/paper/integral-obje...,http://openaccess.thecvf.com/content_ICCV_2019...
91,HistoSegNet: Semantic Segmentation of Histolog...,https://paperswithcode.com/paper/histosegnet-s...,http://openaccess.thecvf.com/content_ICCV_2019...
92,Self-supervised Equivariant Attention Mechanis...,https://paperswithcode.com/paper/self-supervis...,https://arxiv.org/pdf/2004.04581v1.pdf


In [65]:
#os로 파일 다운로드
pdf_url = df['pdf'].values.tolist()
pdf_file = [i.split('/')[-1] for i in df['pdf'].values if 'arxiv' in i]

print(f'전체 파일 수: {len(pdf_url)}')

#enumerate -> index / for n, ( , ) = n=index, ( , )=tuple
#start /b =>os background실행
for n, (url, file) in enumerate(zip(pdf_url, pdf_file)):
    print(n)
    download_path = f'C:/Users/kimEn/Downloads/pdf_download'
    os.system(f'curl "{url}" --output "{download_path}/{file}"')

전체 파일 수: 94
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91


In [67]:
#오류제외 파일변환

for i in tqdm(pdf_file):    
    try:
        text = pdf_to_txt(f'{download_path}/{i}')
        txt_file_name = '.'.join(i.split('.')[:-1])
        with open(f'{download_path}/txt/{txt_file_name}.txt', 'w', encoding='utf-8') as f:
            f.write(text)
    except Exception as e:
        print(e, i)

  0%|          | 0/92 [00:00<?, ?it/s]

No /Root object! - Is this really a PDF? 2004.04581v1.pdf
No /Root object! - Is this really a PDF? 2007.01947v2.pdf


In [138]:

def get_wordnet_pos(tagged_pos):
    for pos in ['V', 'N', 'J', 'R']:
        if tagged_pos.startswith(pos):
            return pos.lower() if pos != 'J' else 'a'
    return None

lemm = WordNetLemmatizer()
word_txt={}

for txt_file in tqdm(glob('./Downloads/pdf_download/txt/*.txt')):
    with open(txt_file, encoding='utf-8') as f:
        txt = f.read().lower().replace('-\n', '')
        
        
    # 토크나이저
        re_tokenizer = RegexpTokenizer('[a-zA-Z]{2,}')

        # 토큰화
        word_tokens = re_tokenizer.tokenize(txt)

        # 불용어(stopwords) 제거
        stop_words = stopwords.words('english')
        stop_words.append('cid')
        word_tokens = [w for w in word_tokens if w not in stop_words]

        #stemmer = PorterStemmer()
        stemmer = SnowballStemmer('english')
        word_tokens = [stemmer.stem(w) for w in word_tokens]

        word_tokens = pos_tag(word_tokens)
        word_tokens = [(w, get_wordnet_pos(tag)) for w, tag in word_tokens if get_wordnet_pos(tag) != None]
        word_tokens = [lemm.lemmatize(word, pos=tag) for word, tag in word_tokens]
        
        if word_txt.values() == None:
            for f in FreqDist(word_tokens):
                word_txt[f]=txt_file



  0%|          | 0/90 [00:00<?, ?it/s]

In [139]:
word_txt

{}

In [71]:
save_keywords=FreqDist(word_tokens)

In [129]:
#{f}가 제일먼저 나오는 text, 출처url


keywords=FreqDist(word_tokens)
terms_url={}
href=[]
terminologies={}
explain=[]
explain_url=[]

driver = webdriver.Chrome('chromedriver.exe', options=options)

for n,f in enumerate(FreqDist(word_tokens)):
    if n<500:
        
        driver.get('https://www.google.com')
        elem=driver.find_element(By.CLASS_NAME, 'gLFyf.gsfi')

        # 검색
        elem.clear()
        elem.send_keys(f'what is {f} of deep learning')
        driver.find_element(By.CLASS_NAME, 'gNO89b').submit()  
#driver.back()                 
        
        explain_elem=driver.find_elements(By.CLASS_NAME, 'yp1CPe.wDYxhc.NFQFxe.viOShc.LKPcQc > div > div > div > div')        
        explain_href_elem=driver.find_elements(By.CLASS_NAME, 'yp1CPe.wDYxhc.NFQFxe.viOShc.LKPcQc > div > div > div > div > div > div > a')
         
        for e in explain_elem:
            explain.append(e.text)
            for eh in explain_href_elem:
                explain_url.append(eh.get_attribute('href'))
                terms_url[eh.get_attribute('href')]=e.text
                if terminologies.get(f)==None:
                    terminologies[f]=e.text

        time.sleep(2)
    

KeyboardInterrupt: 

In [130]:
from googletrans import Translator
from itertools import chain
from collections import defaultdict

Ttranslated={}
translate_result=[]

explain_translation={}
keywords_translation={}

for i in terminologies.values():
    translator=Translator()
    result=translator.translate(i, src='en', dest='ko')
    translate_result.append(result.text)
    explain_translation[i]=result.text #원문-번역 dict
    
keywords_translation=dict(zip(keywords.keys(),translate_result)) #단어-해석 dict

test_keywords_translation = defaultdict(list)

for k, v in chain(keywords.items(), keywords_translation.items()): #dict에서 중복되는거 리스트로 겹치게
    test_keywords_translation[k].append(v)


df2=pd.DataFrame.from_dict(dict(test_keywords_translation), orient='index')
df2

df2=pd.DataFrame.from_dict(dict(enumerate(test_keywords_translation.items())), orient='index')
df2

df2.to_csv("keywords.csv",header=True, index=True, encoding='utf-8')



,Unnamed: 0,0,1
0,0,procedur,"['In deep learning, a computer model learns to..."
1,1,architectur,['The basic deep learning architecture has a f...
2,2,imagenet,['ImageNet is an image database organized acco...
3,3,augment,['Data augmentation is a set of techniques to ...
4,4,model,"['In deep learning, a computer model learns to..."
5,5,differ,"[""Machine learning means computers learning fr..."
6,6,tabl,['Building a deep learning model with TensorFl...
7,7,timm,['`timm` is a deep-learning library created by...
8,8,epoch,['An epoch is a term used in machine learning ...
9,9,paramet,['Model Parameters are properties of training ...


In [9]:
keywords=pd.read_csv('keywords.csv')#header=None)
keywords.rename(columns={'Unnamed: 0':'인덱스(빈도수)','0':'단어', '1':'의미/해석'}, inplace=True )
keywords

,인덱스(빈도수),단어,의미/해석
0,0,procedur,"['In deep learning, a computer model learns to..."
1,1,architectur,['The basic deep learning architecture has a f...
2,2,imagenet,['ImageNet is an image database organized acco...
3,3,augment,['Data augmentation is a set of techniques to ...
4,4,model,"['In deep learning, a computer model learns to..."
5,5,differ,"[""Machine learning means computers learning fr..."
6,6,tabl,['Building a deep learning model with TensorFl...
7,7,timm,['`timm` is a deep-learning library created by...
8,8,epoch,['An epoch is a term used in machine learning ...
9,9,paramet,['Model Parameters are properties of training ...


In [10]:
import os #keyword 들어간 txt의 url 검색


words_txt={}

for txt_file in tqdm(glob('./Downloads/pdf_download/txt/*.txt')):
    with open(txt_file, encoding='utf-8') as f:

        sentence=f.read().lower().replace('-\n', '').replace('\n', ' ').split('.')
        for i in range(len(keywords.loc[0])):
            if keywords.loc[i]
        txt_title.append(txt_file)
        #print(os.system('os.curdir()')) #현재 디렉토리
        words_txt[]=
        print(os.path.basename(txt_file)) #파일명 추출
        

#sentence가 들어있던 txt_title 불러오기: sentence를 사전으로 저장/


SyntaxError: invalid syntax (Temp/ipykernel_15504/3451960984.py, line 4)

In [ ]:
#classification is folllowed by sota
df3=pd.DataFrame(columns=['단어','대분류','소분류'])

df3['단어']=meaning.keys()
df3['대분류']=cla_dict.keys()
df3['소분류']=cla_dict.values()

df3.to_csv("classification.csv",header=True, index=False)
df3_classification=pd.read_csv('classificasion.csv')
df3

In [ ]:
df_result=pd.merge(df2,df3, on='단어', how='left')
print(df_result)

df_result.to_csv("result.csv",header=True, index=False)
df3_classification=pd.read_csv('result.csv')
df3

In [ ]:
#html 출력(table)

cvs_read=pd.read_csv('result.csv')
cvs_read.to_html('terminologies.html')


In [ ]:
# html_text="""
# <!DOCTYPE html>
# <html>
#     <head>
#         <title>Deep Learning Terminologies</title>
#     </head>
#     <body>
#         <h1>딥러닝 개발자 영어 단어</h1>

        
        
#     </body>
# </html>

# """

# html_file=open('terminologies.html'',w) #html파일 생성
# html_file.write(html_text)
# html_file.close()


# # #추가모드
# # with open('html_file.html','a') as html_file:
# #     html_file.write(html_text)

# #사전기능-> 검색 시 검색 결과에 keywords 제대로 있는지 확인/ 없으면 출력x

In [ ]:
#distionary 성능향상